In [1]:
import torch as to
import os

In [2]:
dataset_root = '/Users/ikroosh/Downloads/maildir'

In [3]:
to_one_email_count = 0
emails = set()

for person in os.listdir(dataset_root):
    if not os.path.exists(os.path.join(dataset_root, person, 'sent')):
        continue

    for email_file_name in os.listdir(os.path.join(dataset_root, person, 'sent')):
        with open(os.path.join(dataset_root, person, 'sent', email_file_name)) as f:
            try:
                data = f.readlines()
                result = []
                to_lines = [line for line in data if line.startswith('To')]
                if len(to_lines) == 0:
                    continue

                for email in to_lines[0][4:-1].split(','):
                    if email.strip() != '':
                        result.append(email.strip())

                if len(result) == 1:
                    to_one_email_count += 1 

                emails.update(result)
            except UnicodeDecodeError:
                continue

In [4]:
uniq_emails_count = len(emails)

In [5]:
enron_emails = len([email for email in emails if 'enron.com' in email])
not_enron_emails = uniq_emails_count - enron_emails

In [50]:
list_emails = [e for e in list(emails) if re.match(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", e) is not None]

In [93]:
list_emails = sorted(list_emails)

In [13]:
with open('./emails.txt', 'w') as f:
    for e in list_emails:
        f.write(e + '\n')

In [15]:
import numpy as np

In [94]:
encoded_emails = np.array([np.array([ord(c) for c in e]) for e in list_emails])

In [220]:
max_email_size = 30
encoded_emails_filtered = [np.append(e, np.array([0] * (max_email_size - len(e)))) for e in encoded_emails if len(e) <= max_email_size]
encoded_emails_filtered_labels = np.array(range(len(encoded_emails_filtered)))

In [226]:
train_test_split = 1
percentile90 = int(train_test_split * len(encoded_emails_filtered))
train_encoded_emails = encoded_emails_filtered[:percentile90]
train_encoded_emails_labels = encoded_emails_filtered_labels[:percentile90]
test_encoded_emails = encoded_emails_filtered[percentile90:]
test_encoded_emails_labels = encoded_emails_filtered_labels[percentile90:]

In [227]:
len(test_encoded_emails) + len(train_encoded_emails), len(train_encoded_emails_labels) + len(test_encoded_emails_labels)

(7481, 7481)

In [228]:
from torch.utils import data
from torch.utils.data import Dataset

class StringsDataset(Dataset):
    def __init__(self, emails, labels):
        self.emails = emails
        self.labels = labels

    def __len__(self):
        return len(self.emails)

    def __getitem__(self, num):
        x = to.Tensor(self.emails[num])
        y = self.labels[num]
        sample = {'x': x, 'y': y}

        return sample

In [256]:
import torch
from torch.autograd import Variable
from tensorboardX import SummaryWriter

import time

email_input_size = 30
hidden_size = 500
out_size = len(train_emails_dataset)

batch_size = 50
epochs = 100

model = torch.nn.Sequential(
    torch.nn.Linear(email_input_size, hidden_size),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(hidden_size, hidden_size * 2),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(hidden_size * 2, hidden_size * 4),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(hidden_size * 4, out_size),
)

loss_fn = torch.nn.CrossEntropyLoss()

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

summary_writer = SummaryWriter(log_dir=os.path.join('./logs', str(int(time.time()))))

train_emails_dataset = StringsDataset(train_encoded_emails, train_encoded_emails_labels)
train_loader = data.DataLoader(train_emails_dataset, batch_size=batch_size, shuffle=True)

test_emails_dataset = StringsDataset(test_encoded_emails, train_encoded_emails_labels)
test_loader = data.DataLoader(test_emails_dataset, batch_size=batch_size)


train_total_batches = np.ceil(len(train_emails_dataset) / float(batch_size))

for epoch in range(epochs):
    model.train(True)
    for i, batch in enumerate(train_loader):
        x = Variable(batch['x'])
#         y = Variable(batch['y']).float() # mse
        y = Variable(batch['y'])
        y_pred = model(x)

        # Compute and print loss.
        loss = loss_fn(y_pred, y)
        
#         if i % 1000 == 0:
        print(epoch, i, loss.data[0])

        summary_writer.add_scalar(f'train mse loss', loss.data[0], epoch * train_total_batches + i)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#     val_loss = 0.0
#     batches = 0
#     model.train(False)
#     for i, batch in enumerate(test_loader):
#         x = Variable(batch['x'])
#         y = Variable(batch['y']).float()
#         y_pred = model(x)

#         val_loss += loss_fn(y_pred, y)
#         batches += 1

#     summary_writer.add_scalar(f'test mse loss', val_loss / batches, epoch)

0 0 14.514791488647461
0 1 12.833527565002441
0 2 12.801584243774414
0 3 12.615930557250977
0 4 12.155817031860352
0 5 11.543993949890137
0 6 12.179008483886719
0 7 12.81513786315918
0 8 11.869431495666504
0 9 11.755170822143555
0 10 11.832274436950684
0 11 11.482391357421875
0 12 11.878050804138184
0 13 11.25343132019043
0 14 11.256272315979004
0 15 11.048187255859375
0 16 10.96483039855957
0 17 10.917158126831055
0 18 10.890167236328125
0 19 10.877676010131836
0 20 10.613869667053223
0 21 10.339638710021973
0 22 10.30299186706543
0 23 10.256381034851074
0 24 9.839761734008789
0 25 10.214099884033203
0 26 9.93600845336914
0 27 9.829767227172852
0 28 9.754682540893555
0 29 9.557165145874023
0 30 9.731775283813477
0 31 9.586505889892578
0 32 9.66711139678955
0 33 9.449906349182129
0 34 9.481612205505371
0 35 9.244452476501465
0 36 9.300447463989258
0 37 9.374164581298828
0 38 9.318922996520996
0 39 9.30053424835205
0 40 9.161946296691895
0 41 9.046865463256836
0 42 9.328313827514648
0 4

2 56 6.8011250495910645
2 57 6.936903476715088
2 58 7.014148712158203
2 59 7.057922840118408
2 60 7.136713981628418
2 61 7.051253795623779
2 62 7.064056873321533
2 63 7.078528881072998
2 64 7.007734775543213
2 65 7.061218738555908
2 66 7.0416975021362305
2 67 7.218156814575195
2 68 7.183131217956543
2 69 7.051827907562256
2 70 7.237410068511963
2 71 6.990127086639404
2 72 7.122800350189209
2 73 7.1532673835754395
2 74 7.145440578460693
2 75 7.072679042816162
2 76 6.810397148132324
2 77 6.961596488952637
2 78 6.8760666847229
2 79 7.198736190795898
2 80 7.062728404998779
2 81 7.136657238006592
2 82 7.143408298492432
2 83 6.928451061248779
2 84 6.892803192138672
2 85 7.111997604370117
2 86 6.8869218826293945
2 87 6.836385726928711
2 88 7.263552665710449
2 89 6.888430595397949
2 90 7.137716293334961
2 91 6.982351779937744
2 92 7.2067766189575195
2 93 7.089451789855957
2 94 7.014894008636475
2 95 7.0293378829956055
2 96 7.034459114074707
2 97 7.2415947914123535
2 98 6.773987293243408
2 99 6

4 108 4.906509876251221
4 109 4.901700019836426
4 110 4.603151321411133
4 111 5.053298473358154
4 112 4.721401691436768
4 113 4.854078769683838
4 114 4.627098560333252
4 115 4.56496524810791
4 116 4.786782741546631
4 117 5.069303035736084
4 118 4.890628814697266
4 119 4.618841171264648
4 120 4.531197547912598
4 121 4.4790120124816895
4 122 4.655063152313232
4 123 4.614101409912109
4 124 4.8550825119018555
4 125 4.82907772064209
4 126 4.896592617034912
4 127 4.703812122344971
4 128 4.9636921882629395
4 129 4.951444625854492
4 130 5.021130561828613
4 131 5.145021438598633
4 132 4.757032871246338
4 133 4.523638725280762
4 134 4.769306182861328
4 135 4.8013081550598145
4 136 4.671936988830566
4 137 4.8259172439575195
4 138 4.777900218963623
4 139 4.805192947387695
4 140 4.990454196929932
4 141 4.580219745635986
4 142 4.517444610595703
4 143 4.782444477081299
4 144 5.058879852294922
4 145 4.733011245727539
4 146 4.989643096923828
4 147 4.097379684448242
4 148 4.945943832397461
4 149 4.91650

7 6 3.0959115028381348
7 7 2.6828439235687256
7 8 3.2499589920043945
7 9 2.83731746673584
7 10 2.922832727432251
7 11 3.2019739151000977
7 12 2.6020281314849854
7 13 3.2327394485473633
7 14 2.8735733032226562
7 15 2.839646339416504
7 16 3.2161989212036133
7 17 3.068211078643799
7 18 2.9283270835876465
7 19 2.6413774490356445
7 20 2.6704165935516357
7 21 2.5781545639038086
7 22 2.3644492626190186
7 23 2.9259519577026367
7 24 3.0030453205108643
7 25 3.156149387359619
7 26 2.9637022018432617
7 27 3.3066961765289307
7 28 2.9145522117614746
7 29 3.1648921966552734
7 30 2.8520309925079346
7 31 2.734445095062256
7 32 2.9122889041900635
7 33 2.9685306549072266
7 34 3.2236270904541016
7 35 3.0710010528564453
7 36 3.043367624282837
7 37 3.2657480239868164
7 38 2.9540696144104004
7 39 2.665255069732666
7 40 2.795041084289551
7 41 2.8551764488220215
7 42 2.7867238521575928
7 43 2.862034797668457
7 44 3.2146739959716797
7 45 2.983959436416626
7 46 2.906317710876465
7 47 2.9582419395446777
7 48 3.13

9 52 2.076497793197632
9 53 2.7187998294830322
9 54 2.2488770484924316
9 55 2.494436264038086
9 56 2.4358975887298584
9 57 2.3728232383728027
9 58 2.1052019596099854
9 59 2.7772018909454346
9 60 2.647615671157837
9 61 2.575103759765625
9 62 2.4402801990509033
9 63 2.436868190765381
9 64 2.6926653385162354
9 65 2.208343982696533
9 66 2.3188834190368652
9 67 2.160257577896118
9 68 2.1715269088745117
9 69 2.383730411529541
9 70 2.3768410682678223
9 71 2.6340441703796387
9 72 2.3594248294830322
9 73 2.4467973709106445
9 74 2.6309638023376465
9 75 2.0772948265075684
9 76 2.461099863052368
9 77 2.568392276763916
9 78 2.40769362449646
9 79 2.347639560699463
9 80 2.167654514312744
9 81 2.534707546234131
9 82 2.425990104675293
9 83 2.541640520095825
9 84 2.4631409645080566
9 85 2.357095241546631
9 86 2.4476349353790283
9 87 2.2435965538024902
9 88 2.6623613834381104
9 89 2.6022350788116455
9 90 2.345959186553955
9 91 2.595569372177124
9 92 2.4875171184539795
9 93 2.7747583389282227
9 94 2.38247

...

58 24 0.056627918034791946
58 25 0.17555537819862366
58 26 0.23443832993507385
58 27 0.0389726348221302
58 28 0.015853600576519966
58 29 0.030089104548096657
58 30 0.09096347540616989
58 31 0.05638238042593002
58 32 0.12751905620098114
58 33 0.052984919399023056
58 34 0.083901546895504
58 35 0.11067774891853333
58 36 0.11819319427013397
58 37 0.11764387041330338
58 38 0.031919416040182114
58 39 0.026589149609208107
58 40 0.11600902676582336
58 41 0.04740503430366516
58 42 0.02025214023888111
58 43 0.138926699757576
58 44 0.21766701340675354
58 45 0.10950212180614471
58 46 0.008899550884962082
58 47 0.04555487260222435
58 48 0.016587650403380394
58 49 0.1460934281349182
58 50 0.24836261570453644
58 51 0.08535981923341751
58 52 0.037422873079776764
58 53 0.18733453750610352
58 54 0.2607215344905853
58 55 0.10721629112958908
58 56 0.08578279614448547
58 57 0.3451456129550934
58 58 0.3305785357952118
58 59 0.1673363894224167
58 60 0.19565527141094208
58 61 0.03863000497221947
58 62 0.24759

60 36 0.030010826885700226
60 37 0.011718438938260078
60 38 0.01269110944122076
60 39 0.11953418701887131
60 40 0.21850205957889557
60 41 0.0357230007648468
60 42 0.10897005349397659
60 43 0.014478658325970173
60 44 0.01797611266374588
60 45 0.03989461436867714
60 46 0.010560777969658375
60 47 0.17179147899150848
60 48 0.05635111406445503
60 49 0.06773851811885834
60 50 0.16896596550941467
60 51 0.21989871561527252
60 52 0.0542118176817894
60 53 0.034746378660202026
60 54 0.11303912103176117
60 55 0.33167925477027893
60 56 0.08329636603593826
60 57 0.009591401554644108
60 58 0.13295158743858337
60 59 0.056991882622241974
60 60 0.08721429109573364
60 61 0.083638995885849
60 62 0.005288948304951191
60 63 0.09851553291082382
60 64 0.07077769190073013
60 65 0.04832687973976135
60 66 0.10149196535348892
60 67 0.01953820325434208
60 68 0.08069112151861191
60 69 0.05393423140048981
60 70 0.07515089213848114
60 71 0.03961888700723648
60 72 0.025042543187737465
60 73 0.05819769948720932
60 74 0

62 47 0.011517588049173355
62 48 0.01128485519438982
62 49 0.0866248831152916
62 50 0.040030837059020996
62 51 0.06800971180200577
62 52 0.1474643051624298
62 53 0.01916636899113655
62 54 0.03259716182947159
62 55 0.1339060515165329
62 56 0.14851540327072144
62 57 0.012404050678014755
62 58 0.1049070954322815
62 59 0.03761184588074684
62 60 0.14540010690689087
62 61 0.0974361002445221
62 62 0.03582032769918442
62 63 0.04175136983394623
62 64 0.3432510495185852
62 65 0.02145196869969368
62 66 0.2837156355381012
62 67 0.04490898177027702
62 68 0.034377433359622955
62 69 0.0734298899769783
62 70 0.174089252948761
62 71 0.13355648517608643
62 72 0.16951099038124084
62 73 0.10273744910955429
62 74 0.030024085193872452
62 75 0.16733317077159882
62 76 0.12572169303894043
62 77 0.19027872383594513
62 78 0.07472822070121765
62 79 0.16652707755565643
62 80 0.04360249638557434
62 81 0.040282294154167175
62 82 0.22026169300079346
62 83 0.04603248089551926
62 84 0.1455734223127365
62 85 0.143052697

64 60 0.1495158076286316
64 61 0.09326431900262833
64 62 0.19980210065841675
64 63 0.0645834356546402
64 64 0.07353804260492325
64 65 0.45995986461639404
64 66 0.062379077076911926
64 67 0.09833353012800217
64 68 0.2845138609409332
64 69 0.04041939228773117
64 70 0.01058273296803236
64 71 0.12025424093008041
64 72 0.01625615544617176
64 73 0.08948826789855957
64 74 0.008908724412322044
64 75 0.12036293745040894
64 76 0.13067375123500824
64 77 0.14258433878421783
64 78 0.09202268719673157
64 79 0.04370801895856857
64 80 0.1195145770907402
64 81 0.0113869309425354
64 82 0.04215671867132187
64 83 0.059105273336172104
64 84 0.11486373841762543
64 85 0.04471028223633766
64 86 0.05539153516292572
64 87 0.19356685876846313
64 88 0.12986811995506287
64 89 0.003681743051856756
64 90 0.01756616123020649
64 91 0.3012886345386505
64 92 0.16663441061973572
64 93 0.01216130517423153
64 94 0.048499301075935364
64 95 0.16121941804885864
64 96 0.1169453114271164
64 97 0.1204267144203186
64 98 0.1396433

KeyboardInterrupt: 

In [270]:
''.join([chr(c) for c in encoded_emails_filtered[1234]])

'cliff.feehan@tdsecurities.com\x00'

In [272]:
print(encoded_emails_filtered[1234])
to.nn.Softmax()(model(Variable(to.Tensor(encoded_emails_filtered[4321])))).topk(1)

[ 99 108 105 102 102  46 102 101 101 104  97 110  64 116 100 115 101  99
 117 114 105 116 105 101 115  46  99 111 109   0]


/Users/ikroosh/.pyenv/versions/3.6.1/envs/learned-indices-experiments/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


(Variable containing:
  0.9990
 [torch.FloatTensor of size 1], Variable containing:
  4321
 [torch.LongTensor of size 1])

In [257]:
model()

Sequential(
  (0): Linear(in_features=30, out_features=500)
  (1): LeakyReLU(0.01)
  (2): Linear(in_features=500, out_features=1000)
  (3): LeakyReLU(0.01)
  (4): Linear(in_features=1000, out_features=2000)
  (5): LeakyReLU(0.01)
  (6): Linear(in_features=2000, out_features=7481)
)

In [276]:
from sklearn.cluster import KMeans

In [277]:
KMeans(n_clusters=100, random_state=0).fit(encoded_emails_filtered)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=100, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [278]:
kmeans = _

In [279]:
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=100, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [280]:
list_emails

['05196715@juno.com',
 '1353@aol.com',
 '2740741@skytel.com',
 '2812677295@page.metrocall.com',
 '3973996@skytel.com',
 '3@enron.com',
 '4069450@archwireless.net',
 '5594935@skytel.com',
 '6202354@skytel.com',
 '7028587@skytel.com',
 '7137160795@pagenet.com',
 '7137167243.7160795@pagenet.net',
 '7409948@enron.com',
 '7409949@skytel.com',
 '7409950@skytel.com',
 '8774619489@skytel.com',
 '8774820206@pagenetmessage.net',
 '8774820772@pagenetmessage.net',
 '8774892073@pagenetmessage.net',
 '8774984248@pagenetmessage.net',
 '8777208398.4891939@pagenetmessage.net',
 '8777208398.4891940@pagenetmessage.net',
 '8778552081@pagenetmessage.net',
 '8886424977@pagenetmessage.net',
 '8887107934@pagenetmessage.net',
 '8887107935@pagenetmessage.net',
 '8915473@archwireless.net',
 '9069761@enron.com',
 '9069761@skytel.com',
 '9069876@skytel.com',
 '9069878@skytel.com',
 '9079876@skytel.com',
 '9162262@skytel.com',
 '9167185@skytel.com',
 '9167188@enron.com',
 '9169761@skytel.com',
 'Yrnffin1@aol.com',


In [287]:
ben_j = np.append(np.array([ord(c) for c in 'ben.jacoby@enron.com']), np.array([0] * (max_email_size - len('ben.jacoby@enron.com'))))
ben_h = np.append(np.array([ord(c) for c in 'ben.herman@enron.com']), np.array([0] * (max_email_size - len('ben.herman@enron.com'))))
brian_h = np.append(np.array([ord(c) for c in 'brian.hoskins@enron.com']), np.array([0] * (max_email_size - len('brian.hoskins@enron.com'))))

In [289]:
kmeans.predict([brian_h])

array([15], dtype=int32)

In [301]:
''.join([chr(int(np.round(c))) for c in list(kmeans.cluster_centers_[10])])

'kholk/linikhkffkNgilll/bom\x00\x00\x00\x00'